# Example: Hydropower Generation Simulation

This notebook demonstrates a simple hydropower generation scenario. The system consists of an `Upper Lake` that feeds water through a `Water Turbine`, which then discharges into a `Tailrace Canal`.

This is a physics-based simulation without any active control agents. The simulation runs for 3 days with a 1-hour time step.

## System Components

- **Lake (`upper_lake`)**: A large water body with a defined surface area and volume. It also includes a parameter for evaporation.
- **Water Turbine (`turbine_1`)**: Generates power based on the flow of water passing through it. It has a defined efficiency and maximum flow rate.
- **Canal (`tailrace_canal`)**: A channel that carries the water away from the turbine.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.simulation_identification.physical_objects.lake import Lake
from swp.simulation_identification.physical_objects.water_turbine import WaterTurbine
from swp.simulation_identification.physical_objects.canal import Canal

# 1. Create the simulation harness
# Simulate for 3 days (in seconds) with hourly steps (3600s)
harness = SimulationHarness(config={'dt': 3600, 'duration': 86400 * 3})

# 2. Create physical components
initial_lake_volume = 40e6
lake_surface_area = 2e6
upper_lake = Lake(
    name="upper_lake",
    initial_state={'volume': initial_lake_volume, 'water_level': initial_lake_volume / lake_surface_area, 'outflow': 0},
    parameters={'surface_area': lake_surface_area, 'max_volume': 50e6, 'evaporation_rate_m_per_s': 2.31e-8}
)

turbine = WaterTurbine(
    name="turbine_1",
    initial_state={'power': 0, 'outflow': 0},
    parameters={'efficiency': 0.85, 'max_flow_rate': 150}
)

tailrace_canal = Canal(
    name="tailrace_canal",
    initial_state={'volume': 100000, 'water_level': 2.1, 'outflow': 0},
    parameters={'bottom_width': 20, 'length': 5000, 'slope': 0.0002, 'side_slope_z': 2, 'manning_n': 0.025}
)

# 3. Add components to the harness
harness.add_component(upper_lake)
harness.add_component(turbine)
harness.add_component(tailrace_canal)

# 4. Define the connections between components
harness.add_connection("upper_lake", "turbine_1")
harness.add_connection("turbine_1", "tailrace_canal")

# 5. Build and run the simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_simulation()
sys.stdout = original_stdout

print("Hydropower simulation complete.")

## Results and Visualization

We plot the key performance indicators of the hydropower system:
- The water level of the upper lake.
- The power generated by the turbine.
- The outflow from the turbine.

In [ ]:
# Extract data from history
time_hours = [h['time'] / 3600 for h in harness.history] # Convert time to hours for plotting
lake_levels = [h['upper_lake']['water_level'] for h in harness.history]
turbine_power_mw = [h['turbine_1']['power'] / 1e6 for h in harness.history] # Convert power to Megawatts
turbine_outflow = [h['turbine_1']['outflow'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time (hours)': time_hours,
    'Lake Level (m)': lake_levels,
    'Turbine Power (MW)': turbine_power_mw,
    'Turbine Outflow (m^3/s)': turbine_outflow
})

print(df.head())

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Lake Level
ax1.plot(df['Time (hours)'], df['Lake Level (m)'], label='Lake Water Level', color='blue')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Hydropower Simulation Results (3 Days)')
ax1.grid(True)
ax1.legend()

# Turbine Power
ax2.plot(df['Time (hours)'], df['Turbine Power (MW)'], label='Turbine Power', color='green')
ax2.set_ylabel('Power (MW)')
ax2.grid(True)
ax2.legend()

# Turbine Outflow
ax3.plot(df['Time (hours)'], df['Turbine Outflow (m^3/s)'], label='Turbine Outflow', color='red')
ax3.set_xlabel('Time (hours)')
ax3.set_ylabel('Outflow (m^3/s)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()